In [0]:
ecommerce_data_2019_nov_df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True)
display(ecommerce_data_2019_nov_df.limit(3))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,null,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387


In [0]:
ecommerce_data_2019_oct_df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True)
display(ecommerce_data_2019_oct_df.limit(3))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8


In [0]:
common_columns = ["product_id", "category_id", "user_id"]
# Inner Join
inner_join_df = ecommerce_data_2019_nov_df.join(ecommerce_data_2019_oct_df[[common_columns]], on = common_columns, how = "inner")
print(f"{inner_join_df.count()}, {len(inner_join_df.columns)}")
# Left Outer Join
left_join_df = ecommerce_data_2019_nov_df.join(ecommerce_data_2019_oct_df[[common_columns]], on = common_columns, how = "left")
print(f"{left_join_df.count()}, {len(left_join_df.columns)}")
# Right Outer Join
right_join_df = ecommerce_data_2019_nov_df.join(ecommerce_data_2019_oct_df[[common_columns]], on = common_columns, how = "right")
print(f"{right_join_df.count()}, {len(right_join_df.columns)}")
# Full Outer Join
outer_join_df = ecommerce_data_2019_nov_df.join(ecommerce_data_2019_oct_df[[common_columns]], on = common_columns, how = "outer")
print(f"{outer_join_df.count()}, {len(outer_join_df.columns)}")

25294532, 9
88065478, 9
63379150, 9
126150096, 9


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum, asc, desc, round, coalesce, lit
# Selecting only few columns
inner_join_mini_df = inner_join_df.select("user_id", "event_time", "product_id", "brand", "price")
# Converting price column to numeric and handling nulls
inner_join_mini_intprice_df = inner_join_mini_df.withColumn("price_num", coalesce(col("price").cast("double"), lit(0)))
# Calculating running total of price, partitioned by user_id, ordered by event_time
inner_join_mini_intprice_running_total_df = inner_join_mini_intprice_df.withColumn(
    "running_total_price",
    sum("price_num").over(Window.partitionBy("user_id").orderBy("event_time"))
)
# Displaying the results
display(inner_join_mini_intprice_running_total_df.limit(25))

user_id,event_time,product_id,brand,price,price_num,running_total_price
216064734,2019-11-17 20:24:03 UTC,7003373,bugaboo,818.48,818.48,2455.44
216064734,2019-11-17 20:24:03 UTC,7003373,bugaboo,818.48,818.48,2455.44
216064734,2019-11-17 20:24:03 UTC,7003373,bugaboo,818.48,818.48,2455.44
266287781,2019-11-21 09:48:22 UTC,6000157,starline,95.76,95.76,95.76
266287781,2019-11-21 09:49:01 UTC,6000157,starline,95.76,95.76,191.52
298097477,2019-11-21 06:20:42 UTC,1002544,apple,488.82,488.82,977.64
298097477,2019-11-21 06:20:42 UTC,1002544,apple,488.82,488.82,977.64
335470066,2019-11-07 10:22:02 UTC,26402230,null,411.09,411.09,411.09
335470066,2019-11-07 10:31:54 UTC,26402230,null,411.09,411.09,822.18
335470066,2019-11-07 10:32:24 UTC,26402230,null,411.09,411.09,1233.27


In [0]:
from pyspark.sql.functions import when
# Create derived feature "value" based on price_num value
inner_join_mini_ip_rt_value_df = inner_join_mini_intprice_running_total_df.withColumn("value", when(col("price_num") >= 2000, "premium").otherwise("regular"))
display(inner_join_mini_ip_rt_value_df.limit(25))

user_id,event_time,product_id,brand,price,price_num,running_total_price,value
216064734,2019-11-17 20:24:03 UTC,7003373,bugaboo,818.48,818.48,2455.44,regular
216064734,2019-11-17 20:24:03 UTC,7003373,bugaboo,818.48,818.48,2455.44,regular
216064734,2019-11-17 20:24:03 UTC,7003373,bugaboo,818.48,818.48,2455.44,regular
266287781,2019-11-21 09:48:22 UTC,6000157,starline,95.76,95.76,95.76,regular
266287781,2019-11-21 09:49:01 UTC,6000157,starline,95.76,95.76,191.52,regular
298097477,2019-11-21 06:20:42 UTC,1002544,apple,488.82,488.82,977.64,regular
298097477,2019-11-21 06:20:42 UTC,1002544,apple,488.82,488.82,977.64,regular
335470066,2019-11-07 10:22:02 UTC,26402230,null,411.09,411.09,411.09,regular
335470066,2019-11-07 10:31:54 UTC,26402230,null,411.09,411.09,822.18,regular
335470066,2019-11-07 10:32:24 UTC,26402230,null,411.09,411.09,1233.27,regular
